### Chess Game with Human vs AI

#### **Overview:**
This Python-based chess game enables a user to play against an AI using the Alpha Beta Pruning algorithm. The game supports basic chess functionalities like moves, castling, pawn promotion, and en passant. The user interface is implemented using Pygame, while the chess logic is modularized into classes for pieces, moves, the game board, and the players.

#### **Key Features:**
1. **Graphical User Interface (GUI)**: 
   - Uses Pygame for rendering the chessboard, pieces, and move history.
   - Real-time interaction with the board for move selection and input.

2. **Piece Classes**: 
   - Classes for each type of chess piece (`King`, `Queen`, `Rook`, `Bishop`, `Knight`, `Pawn`) that define valid moves based on chess rules.

3. **Move Management**:
   - A `Move` class tracks the start and end positions of a piece and handles special moves like castling and pawn promotion.

4. **AI**:
   - An AI opponent is implemented using the Minimax algorithm with alpha-beta pruning to choose optimal moves based on the board state.

5. **Game Flow**:
   - The game alternates between the human player's and the AI’s turns.
   - Game-over conditions (checkmate, stalemate) are checked after each move.

6. **History Tracking**:
   - The move history is recorded and displayed in the GUI. Only the most recent 10 moves are shown for a clean user experience.

#### **Usage:**
- The user can click on the chessboard or input in the text box to select and move pieces. The AI automatically plays its turn after the human player’s move.
- The game ends when checkmate or stalemate occurs, and the result is displayed on the screen.

#### **Technical Details:**
- **Libraries Used**: Pygame for the graphical interface, and Python’s built-in libraries for core functionality.
- **Minimax Algorithm**: Used for AI decision-making with alpha-beta pruning to optimize move selection.

---

In [2]:
import pygame
import sys
import os
import copy
import math
from abc import ABC, abstractmethod
from tkinter import Tk, Canvas, Button, Label, Entry, messagebox
import uuid

# Constants
WIDTH, HEIGHT = 1000, 720
DIMENSION = 8
SQ_SIZE = HEIGHT // DIMENSION
FPS = 30
LIGHT_BROWN = (240, 180, 130)
DARK_BROWN = (180, 110, 50)
TEXT_COLOR = (10, 10, 10)
HISTORY_WIDTH = 280

pygame.init()
FONT = pygame.font.SysFont("Arial", 18)
HISTORY_FONT = pygame.font.SysFont("Arial", 20)

# Load PNG images from the assets folder
PIECES = {}
for color in ['w', 'b']:
    for piece in ['K', 'Q', 'R', 'B', 'N', 'P']:
        img = pygame.image.load(f'assets/{color}{piece}.png')
        PIECES[f'{color}{piece}'] = pygame.transform.scale(img, (SQ_SIZE, SQ_SIZE))

piece_values = {'K': 0, 'Q': 9, 'R': 5, 'B': 3, 'N': 3, 'P': 1}

def evaluate_board(board, ai_color='b'):
    score = 0
    for row in board:
        for piece in row:
            if piece:
                value = piece_values[piece.name.upper()]
                score += value if piece.color == ai_color else -value
    return score

# --- PIECES ---
class Piece(ABC):
    def __init__(self, color):
        self.color = color
        self.has_moved = False

    @abstractmethod
    def get_valid_moves(self, board, r, c):
        pass

    def draw(self, screen, r, c):
        screen.blit(PIECES[f'{self.color}{self.name}'], (c * SQ_SIZE, r * SQ_SIZE))

class King(Piece):
    name = 'K'
    def get_valid_moves(self, board, r, c):
        moves = []
        # Standard king moves
        for dr in [-1, 0, 1]:
            for dc in [-1, 0, 1]:
                if dr == 0 and dc == 0: continue
                nr, nc = r + dr, c + dc
                if 0 <= nr < 8 and 0 <= nc < 8:
                    if board.board[nr][nc] is None or board.board[nr][nc].color != self.color:
                        moves.append((nr, nc))

        # Castling moves (validated separately to avoid recursion)
        if not self.has_moved:
            if board.can_castle(self.color, kingside=True):
                moves.append((r, c + 2))
            if board.can_castle(self.color, kingside=False):
                moves.append((r, c - 2))

        return moves

class Queen(Piece):
    name = 'Q'
    def get_valid_moves(self, board, r, c):
        return Rook(self.color).get_valid_moves(board, r, c) + Bishop(self.color).get_valid_moves(board, r, c)

class Rook(Piece):
    name = 'R'
    def get_valid_moves(self, board, r, c):
        return self._linear_moves(board, r, c, [(1,0), (-1,0), (0,1), (0,-1)])

    def _linear_moves(self, board, r, c, directions):
        moves = []
        for dr, dc in directions:
            for i in range(1, 8):
                nr, nc = r + dr*i, c + dc*i
                if 0 <= nr < 8 and 0 <= nc < 8:
                    if board.board[nr][nc] is None:
                        moves.append((nr, nc))
                    elif board.board[nr][nc].color != self.color:
                        moves.append((nr, nc))
                        break
                    else:
                        break
        return moves

class Bishop(Piece):
    name = 'B'
    def get_valid_moves(self, board, r, c):
        return self._linear_moves(board, r, c, [(1,1), (-1,1), (1,-1), (-1,-1)])

    def _linear_moves(self, board, r, c, directions):
        moves = []
        for dr, dc in directions:
            for i in range(1, 8):
                nr, nc = r + dr*i, c + dc*i
                if 0 <= nr < 8 and 0 <= nc < 8:
                    if board.board[nr][nc] is None:
                        moves.append((nr, nc))
                    elif board.board[nr][nc].color != self.color:
                        moves.append((nr, nc))
                        break
                    else:
                        break
        return moves

class Knight(Piece):
    name = 'N'
    def get_valid_moves(self, board, r, c):
        moves = []
        for dr, dc in [(-2,-1), (-2,1), (-1,-2), (-1,2), (1,-2), (1,2), (2,-1), (2,1)]:
            nr, nc = r+dr, c+dc
            if 0 <= nr < 8 and 0 <= nc < 8:
                if board.board[nr][nc] is None or board.board[nr][nc].color != self.color:
                    moves.append((nr, nc))
        return moves

class Pawn(Piece):
    name = 'P'
    def get_valid_moves(self, board, r, c):
        moves = []
        direction = -1 if self.color == 'w' else 1
        start_row = 6 if self.color == 'w' else 1
        promotion_row = 0 if self.color == 'w' else 7

        # Forward moves
        if 0 <= r + direction < 8 and board.board[r + direction][c] is None:
            if r + direction == promotion_row:
                moves.append((r + direction, c, 'N'))  # Promotion to Knight
            else:
                moves.append((r + direction, c))
            # Double move from starting position
            if r == start_row and board.board[r + 2 * direction][c] is None:
                moves.append((r + 2 * direction, c))

        # Captures
        for dc in [-1, 1]:
            nc = c + dc
            if 0 <= nc < 8 and 0 <= r + direction < 8:
                target = board.board[r + direction][nc]
                if target and target.color != self.color:
                    if r + direction == promotion_row:
                        moves.append((r + direction, nc, 'N'))  # Promotion to Knight
                    else:
                        moves.append((r + direction, nc))
                # En passant
                if r == (4 if self.color == 'w' else 3) and board.last_move:
                    last_move = board.last_move
                    if (isinstance(last_move.piece, Pawn) and 
                        abs(last_move.start[0] - last_move.end[0]) == 2 and
                        last_move.end[0] == r and last_move.end[1] == nc):
                        moves.append((r + direction, nc, 'EP'))

        return moves

class Move:
    def __init__(self, start, end, piece, is_castling=False, rook_start=None, rook_end=None, promotion=None, is_en_passant=False):
        self.start = start
        self.end = end
        self.piece = piece
        self.is_castling = is_castling
        self.rook_start = rook_start
        self.rook_end = rook_end
        self.promotion = promotion
        self.is_en_passant = is_en_passant

class Board:
    def __init__(self):
        self.board = [[None] * 8 for _ in range(8)]
        self.setup_board()
        self.last_move = None

    def setup_board(self):
        self.board[0] = [Rook('b'), Knight('b'), Bishop('b'), Queen('b'), King('b'), Bishop('b'), Knight('b'), Rook('b')]
        self.board[1] = [Pawn('b') for _ in range(8)]
        self.board[6] = [Pawn('w') for _ in range(8)]
        self.board[7] = [Rook('w'), Knight('w'), Bishop('w'), Queen('w'), King('w'), Bishop('w'), Knight('w'), Rook('w')]

    def draw(self, screen, selected=None, valid_moves=[]):
        for r in range(8):
            for c in range(8):
                color = LIGHT_BROWN if (r + c) % 2 == 0 else DARK_BROWN
                pygame.draw.rect(screen, color, pygame.Rect(c * SQ_SIZE, r * SQ_SIZE, SQ_SIZE, SQ_SIZE))

        if selected:
            r, c = selected
            s = pygame.Surface((SQ_SIZE, SQ_SIZE), pygame.SRCALPHA)
            s.fill((255, 255, 0, 80))
            screen.blit(s, (c * SQ_SIZE, r * SQ_SIZE))

        for move in valid_moves:
            r, c = move[0], move[1]
            s = pygame.Surface((SQ_SIZE, SQ_SIZE), pygame.SRCALPHA)
            s.fill((0, 255, 0, 80))
            screen.blit(s, (c * SQ_SIZE, r * SQ_SIZE))

        for r in range(8):
            for c in range(8):
                piece = self.board[r][c]
                if piece:
                    piece.draw(screen, r, c)

    def move_piece(self, move):
        move.piece.has_moved = True
        if move.is_castling:
            self.board[move.end[0]][move.end[1]] = move.piece
            self.board[move.start[0]][move.start[1]] = None
            rook = self.board[move.rook_start[0]][move.rook_start[1]]
            rook.has_moved = True
            self.board[move.rook_end[0]][move.rook_end[1]] = rook
            self.board[move.rook_start[0]][move.rook_start[1]] = None
        elif move.is_en_passant:
            self.board[move.end[0]][move.end[1]] = move.piece
            self.board[move.start[0]][move.start[1]] = None
            # Remove the captured pawn
            self.board[move.start[0]][move.end[1]] = None
        else:
            if move.promotion:
                self.board[move.end[0]][move.end[1]] = Knight(move.piece.color)
            else:
                self.board[move.end[0]][move.end[1]] = move.piece
            self.board[move.start[0]][move.start[1]] = None
        self.last_move = move

    def locate_king(self, color):
        for r in range(8):
            for c in range(8):
                piece = self.board[r][c]
                if piece and piece.color == color and piece.name == 'K':
                    return r, c
        return None

    def in_check(self, color, exclude_king=True):
        enemy = 'b' if color == 'w' else 'w'
        king_pos = self.locate_king(color)
        if not king_pos:
            return True
        kr, kc = king_pos
        for r in range(8):
            for c in range(8):
                piece = self.board[r][c]
                if piece and piece.color == enemy and (not exclude_king or piece.name != 'K'):
                    if (kr, kc) in [move[:2] for move in piece.get_valid_moves(self, r, c)]:
                        return True
        return False

    def can_castle(self, color, kingside=True):
        row = 7 if color == 'w' else 0
        king = self.board[row][4]
        if not king or king.has_moved or self.in_check(color):
            return False

        rook_col = 7 if kingside else 0
        check_cols = [5, 6] if kingside else [1, 2, 3]
        rook = self.board[row][rook_col]
        if not rook or rook.has_moved:
            return False

        for col in check_cols:
            if self.board[row][col] is not None:
                return False

        king_steps = [4, 5] if kingside else [4, 3, 2]
        for col in king_steps:
            test_board = copy.deepcopy(self)
            test_board.board[row][col] = king
            test_board.board[row][4] = None
            if test_board.in_check(color, exclude_king=True):
                return False

        return True

    def get_all_moves(self, color):
        moves = []
        for r in range(8):
            for c in range(8):
                piece = self.board[r][c]
                if piece and piece.color == color:
                    for move in piece.get_valid_moves(self, r, c):
                        is_castling = False
                        rook_start = None
                        rook_end = None
                        promotion = None
                        is_en_passant = False
                        end = move[:2]
                        if len(move) > 2:
                            if move[2] == 'N':
                                promotion = 'N'
                            elif move[2] == 'EP':
                                is_en_passant = True
                        if piece.name == 'K' and abs(c - end[1]) == 2:
                            is_castling = True
                            row = 7 if color == 'w' else 0
                            if end[1] > c:
                                rook_start = (row, 7)
                                rook_end = (row, 5)
                            else:
                                rook_start = (row, 0)
                                rook_end = (row, 3)
                        move_obj = Move((r, c), end, piece, is_castling, rook_start, rook_end, promotion, is_en_passant)
                        test_board = copy.deepcopy(self)
                        test_board.move_piece(move_obj)
                        if not test_board.in_check(color):
                            moves.append(move_obj)
        return moves

    def is_checkmate(self, color):
        return self.in_check(color) and len(self.get_all_moves(color)) == 0

    def is_stalemate(self, color):
        return not self.in_check(color) and len(self.get_all_moves(color)) == 0

    def is_draw(self):
        return self.is_stalemate('w') or self.is_stalemate('b')

# --- PLAYERS ---
class Player(ABC):
    def __init__(self, color):
        self.color = color

class HumanPlayer(Player):
    def __init__(self, color):
        super().__init__(color)
        self.selected = None
        self.start = None
        self.valid_moves = []
        self.text_input_active = False
        self.input_text = ''
        self.input_rect = pygame.Rect(10, HEIGHT - 40, 200, 30)

    def parse_square(self, s):
        if len(s) != 2:
            return None
        file, rank = s[0], s[1]
        if not file.isalpha() or not rank.isdigit():
            return None
        col = ord(file.lower()) - ord('a')
        row = 8 - int(rank)
        if 0 <= row < 8 and 0 <= col < 8:
            return (row, col)
        return None

    def parse_text_move(self, board):
        input_text = self.input_text.strip().lower().replace(' ', '')
        if input_text in ['oo', 'o-o']:
            start = (7, 4) if self.color == 'w' else (0, 4)
            end = (7, 6) if self.color == 'w' else (0, 6)
            piece = board.board[start[0]][start[1]]
            if piece and piece.name == 'K' and board.can_castle(self.color, kingside=True):
                return Move(start, end, piece, is_castling=True, rook_start=(start[0], 7), rook_end=(start[0], 5))
            return None
        elif input_text in ['ooo', 'o-o-o']:
            start = (7, 4) if self.color == 'w' else (0, 4)
            end = (7, 2) if self.color == 'w' else (0, 2)
            piece = board.board[start[0]][start[1]]
            if piece and piece.name == 'K' and board.can_castle(self.color, kingside=False):
                return Move(start, end, piece, is_castling=True, rook_start=(start[0], 0), rook_end=(start[0], 3))
            return None

        if len(input_text) != 4:
            return None
        start_str, end_str = input_text[:2], input_text[2:]
        start = self.parse_square(start_str)
        end = self.parse_square(end_str)
        if not start or not end:
            return None
        start_row, start_col = start
        end_row, end_col = end

        piece = board.board[start_row][start_col]
        if not piece or piece.color != self.color:
            return None

        valid_moves = piece.get_valid_moves(board, start_row, start_col)
        promotion = None
        is_en_passant = False
        for move in valid_moves:
            if move[:2] == (end_row, end_col):
                if len(move) > 2:
                    if move[2] == 'N':
                        promotion = 'N'
                    elif move[2] == 'EP':
                        is_en_passant = True
                break
        else:
            return None

        is_castling = False
        rook_start = None
        rook_end = None
        if piece.name == 'K' and abs(start_col - end_col) == 2:
            is_castling = True
            if end_col > start_col:
                rook_start = (start_row, 7)
                rook_end = (start_row, 5)
            else:
                rook_start = (start_row, 0)
                rook_end = (start_row, 3)

        test_board = copy.deepcopy(board)
        move = Move((start_row, start_col), (end_row, end_col), piece, is_castling, rook_start, rook_end, promotion, is_en_passant)
        test_board.move_piece(move)
        if test_board.in_check(self.color):
            return None
        return move

    def get_move(self, board):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                x, y = pygame.mouse.get_pos()
                if self.input_rect.collidepoint(x, y):
                    self.text_input_active = True
                else:
                    self.text_input_active = False
                    row, col = y // SQ_SIZE, x // SQ_SIZE
                    if self.selected:
                        for move in self.valid_moves:
                            if move[:2] == (row, col):
                                is_castling = False
                                rook_start = None
                                rook_end = None
                                promotion = None
                                is_en_passant = False
                                if len(move) > 2:
                                    if move[2] == 'N':
                                        promotion = 'N'
                                    elif move[2] == 'EP':
                                        is_en_passant = True
                                if self.selected.name == 'K' and abs(self.start[1] - col) == 2:
                                    is_castling = True
                                    if col > self.start[1]:
                                        rook_start = (self.start[0], 7)
                                        rook_end = (self.start[0], 5)
                                    else:
                                        rook_start = (self.start[0], 0)
                                        rook_end = (self.start[0], 3)
                                move_obj = Move(self.start, (row, col), self.selected, is_castling, rook_start, rook_end, promotion, is_en_passant)
                                test_board = copy.deepcopy(board)
                                test_board.move_piece(move_obj)
                                if not test_board.in_check(self.color):
                                    self.selected = None
                                    self.valid_moves = []
                                    return move_obj
                        self.selected = None
                        self.valid_moves = []
                    else:
                        piece = board.board[row][col]
                        if piece and piece.color == self.color:
                            self.selected = piece
                            self.start = (row, col)
                            self.valid_moves = piece.get_valid_moves(board, row, col)
                        else:
                            self.selected = None
                            self.valid_moves = []
            elif event.type == pygame.KEYDOWN and self.text_input_active:
                if event.key == pygame.K_RETURN:
                    move = self.parse_text_move(board)
                    self.text_input_active = False
                    self.input_text = ''
                    if move:
                        return move
                elif event.key == pygame.K_BACKSPACE:
                    self.input_text = self.input_text[:-1]
                else:
                    self.input_text += event.unicode
        return None

class AIPlayer(Player):
    def get_best_move(self, board):
        def minimax(b, depth, alpha, beta, maximizing):
            if depth == 0 or b.is_checkmate(self.color) or b.is_stalemate(self.color):
                return evaluate_board(b.board, ai_color=self.color), None
            color = self.color if maximizing else ('w' if self.color == 'b' else 'b')
            best_move = None
            moves = b.get_all_moves(color)
            moves.sort(key=lambda m: piece_values[m.piece.name.upper()], reverse=maximizing)
            if maximizing:
                max_eval = -math.inf
                for move in moves:
                    new_b = copy.deepcopy(b)
                    new_b.move_piece(move)
                    eval, _ = minimax(new_b, depth - 1, alpha, beta, False)
                    if eval > max_eval:
                        max_eval, best_move = eval, move
                    alpha = max(alpha, eval)
                    if beta <= alpha: break
                return max_eval, best_move
            else:
                min_eval = math.inf
                for move in moves:
                    new_b = copy.deepcopy(b)
                    new_b.move_piece(move)
                    eval, _ = minimax(new_b, depth - 1, alpha, beta, True)
                    if eval < min_eval:
                        min_eval, best_move = eval, move
                    beta = min(beta, eval)
                    if beta <= alpha: break
                return min_eval, best_move
        _, move = minimax(board, 3, -math.inf, math.inf, True)
        return move

# --- MAIN GAME ---
class ChessGame:
    def __init__(self):
        self.screen = pygame.display.set_mode((WIDTH, HEIGHT))
        pygame.display.set_caption("Human vs AI Chess")
        self.clock = pygame.time.Clock()
        self.board = Board()
        self.human = HumanPlayer('w')
        self.ai = AIPlayer('b')
        self.turn = 'w'
        self.running = True
        self.move_history = []
        self.human.input_rect = pygame.Rect(720 + 10, HEIGHT - 40, 200, 30)

    def get_move_notation(self, move, color):
        piece = move.piece
        start_r, start_c = move.start
        end_r, end_c = move.end

        start_file = chr(start_c + 97)
        start_rank = 8 - start_r
        end_file = chr(end_c + 97)
        end_rank = 8 - end_r

        color_prefix = 'White' if color == 'w' else 'Black'
        if move.is_castling:
            if end_c > start_c:
                return f"{color_prefix}: O-O"
            else:
                return f"{color_prefix}: O-O-O"
        notation = f"{color_prefix}: {piece.name}{start_file}{start_rank}-{end_file}{end_rank}"
        if move.promotion:
            notation += "=N"
        if move.is_en_passant:
            notation += " e.p."
        return notation

    def draw_history(self):
        panel_x = HEIGHT
        pygame.draw.rect(self.screen, (255, 255, 255), (panel_x, 0, HISTORY_WIDTH, HEIGHT))
        
        title = HISTORY_FONT.render("Move History", True, (0, 0, 0))
        self.screen.blit(title, (panel_x + 10, 10))

        # Limit the history to the latest 10 moves
        max_lines = min(10, len(self.move_history))
        start_index = len(self.move_history) - max_lines
        
        y = 50
        for i in range(start_index, len(self.move_history)):
            text = HISTORY_FONT.render(self.move_history[i], True, (0, 0, 0))
            self.screen.blit(text, (panel_x + 10, y))
            y += 30


    def display_result(self, message):
        self.screen.fill((0, 0, 0))
        label = FONT.render(message, True, (255, 255, 255))
        self.screen.blit(label, (WIDTH // 2 - 150, HEIGHT // 2))
        pygame.display.flip()
        pygame.time.wait(4000)
        pygame.quit()
        sys.exit()

    def main_loop(self):
        while self.running:
            self.clock.tick(FPS)
            self.screen.fill((0, 0, 0))
            self.board.draw(self.screen, selected=self.human.start, valid_moves=self.human.valid_moves)
            self.draw_history()
            
            pygame.draw.rect(self.screen, (255, 255, 255), self.human.input_rect)
            txt_surface = FONT.render(self.human.input_text, True, (0, 0, 0))
            self.screen.blit(txt_surface, (self.human.input_rect.x + 5, self.human.input_rect.y + 5))
            pygame.draw.rect(self.screen, (0, 0, 0), self.human.input_rect, width=2)

            pygame.display.flip()

            if self.board.is_checkmate(self.turn):
                self.display_result(f"Checkmate! {'White' if self.turn == 'b' else 'Black'} wins.")
            elif self.board.is_stalemate(self.turn):
                self.display_result("Stalemate! It's a draw.")

            move = None
            if self.turn == 'w':  # Human's turn
                move = self.human.get_move(self.board)
            elif self.turn == 'b':  # AI's turn
                move = self.ai.get_best_move(self.board)

            if move:
                current_turn = self.turn  # Save the current turn before switching
                self.board.move_piece(move)  # Apply the move to the board
                opponent_color = 'b' if current_turn == 'w' else 'w'  # Determine opponent

                # Check for check and checkmate
                is_check = self.board.in_check(opponent_color)
                is_checkmate = self.board.is_checkmate(opponent_color)

                # Generate move notation
                move_str = self.get_move_notation(move, current_turn)
                if is_checkmate:
                    move_str += ' - CHECKMATE'
                elif is_check:
                    move_str += ' - CHECK'
                self.move_history.append(move_str)

                # Switch turns
                self.turn = opponent_color


if __name__ == '__main__':
    ChessGame().main_loop()

SystemExit: 